In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd

plt.style.use('fivethirtyeight')
sns.set_context("notebook")

Now let's play around a bit with the large baby names dataset we saw in lecture 1. We'll start by loading that dataset from the social security administration's website.

To keep the data small enough to avoid crashing datahub, we're going to look at only California rather than looking at the national dataset.

In [2]:
import urllib.request
import os.path
import zipfile

data_url = "https://www.ssa.gov/oact/babynames/state/namesbystate.zip"
local_filename = "babynamesbystate.zip"
if not os.path.exists(local_filename): # if the data exists don't download again
    with urllib.request.urlopen(data_url) as resp, open(local_filename, 'wb') as f:
        f.write(resp.read())

zf = zipfile.ZipFile(local_filename, 'r')

ca_name = 'CA.TXT'
field_names = ['State', 'Sex', 'Year', 'Name', 'Count']
with zf.open(ca_name) as fh:
    babynames = pd.read_csv(fh, header=None, names=field_names)

babynames.sample(5)

,State,Sex,Year,Name,Count
44592,CA,F,1961,Dixie,23
157567,CA,F,2002,Jasmyn,32
248205,CA,M,1942,Cliff,15
8679,CA,F,1927,Laurel,26
156511,CA,F,2001,Saniya,5


## Goal 1: Find the most popular baby name in California in 2018

In [3]:
babynames[babynames["Year"] == 2018].sort_values(by = "Count", ascending = False).head(5)

,State,Sex,Year,Name,Count
221160,CA,F,2018,Emma,2743
385701,CA,M,2018,Noah,2569
221161,CA,F,2018,Mia,2499
221162,CA,F,2018,Olivia,2465
385702,CA,M,2018,Liam,2413


## Goal 2: Baby names that start with j. 

In [4]:
starts_with_j = babynames["Name"].str.startswith('J')
starts_with_j.head(10)

0    False
1    False
2    False
3    False
4    False
5    False
6    False
7    False
8    False
9    False
Name: Name, dtype: bool

In [5]:
pd.DataFrame(babynames[starts_with_j]["Name"].unique()).sample(10)

,0
991,Josh
1505,Jaxin
947,Jehilyn
1255,Jese
673,Jisel
791,Josefine
828,Jayanna
678,Jissell
631,Johnae
14,Jacqueline


## Goal 3: Sort the DataFrame by the length of the names.

Approach 1a: Create a new series of only the lengths. Then add that series to the dataframe as a column. Then sort by that column. Then drop that column.

In [11]:
#create a new series of only the lengths
babyname_lengths = babynames["Name"].map(len)

#add that series to the dataframe as a column
babynames["name_length"] = babyname_lengths

#sort by that column
babynames_by_length = babynames.sort_values(by = "name_length")

#drop that column
babynames_by_length = babynames_by_length.drop("name_length", axis=1)
babynames_by_length.head(5)

/tmp/ipykernel_627/3884677794.py:11: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  babynames_by_length = babynames_by_length.drop("name_length", 1)


,State,Sex,Year,Name,Count,name_lengths
85197,CA,F,1980,Ai,6,2
95798,CA,F,1984,Ly,8,2
309713,CA,M,1989,Ed,5,2
334020,CA,M,1999,Jr,9,2
339817,CA,M,2001,Ed,5,2


Approach 1b: Same as 1a, but use str.len() to generate the lengths of the strings.

In [9]:
#create a new series of only the lengths
babyname_lengths = babynames["Name"].str.len()

#add that series to the dataframe as a column
babynames["name_length"] = babyname_lengths

#sort by that column
babynames_by_length = babynames.sort_values(by = "name_length")

#drop that column
babynames_by_length = babynames_by_length.drop("name_length", axis=1)
babynames_by_length.head(5)

,State,Sex,Year,Name,Count,name_lengths
85197,CA,F,1980,Ai,6,2
95798,CA,F,1984,Ly,8,2
309713,CA,M,1989,Ed,5,2
334020,CA,M,1999,Jr,9,2
339817,CA,M,2001,Ed,5,2


Approach 2: Generate an index that is in the order we want. Pass that index to loc.

In [10]:
babynames.loc[babynames["Name"].str.len().sort_values().index].head(5)

,State,Sex,Year,Name,Count,name_lengths,name_length
85197,CA,F,1980,Ai,6,2,2
95798,CA,F,1984,Ly,8,2,2
309713,CA,M,1989,Ed,5,2,2
334020,CA,M,1999,Jr,9,2,2
339817,CA,M,2001,Ed,5,2,2


How does this work exactly? Let's break it into pieces.

In [ ]:
lengths_sorted_by_length = babynames["Name"].str.len().sort_values()
lengths_sorted_by_length.head(5)

In [ ]:
index_sorted_by_length = lengths_sorted_by_length.index
index_sorted_by_length

In [ ]:
# now pass the index to loc. This is yet another way 
# that loc can be used that we did not discuss in lecture!
babynames.loc[index_sorted_by_length].head(5)